<img src="./img/logos_combined.png" align="right" width="50%"></img>
<br><br><br>

# Web Coverage Processing Service (WCPS) - Introduction and example

<hr>
<a href="./01_owslib_example.ipynb"><< OWSlib example</a>&nbsp;<space>&nbsp;<space>&nbsp;<space>&nbsp;<space>&nbsp;<space>&nbsp;<space>&nbsp;<space>&nbsp;<space>&nbsp;<space>&nbsp;<space> <a href="./03_es_data_lib.ipynb">es-data-lib example >></a>

## WCPS request

A **Web Coverage Processing Service** is an extension supported by the WCS 2.0 core specification and can optionally be implemented by a WCS server. WCPS allows for crafting queries to be run on the data using a text based query language, similar to SQL.  
The extension specifies an additional WCS request:  
> the ***ProcessCoverages*** request returns a coverage encoded in a specified format (e.g. netCDF or csv) and allows for mathematical calculations on the requested coverages. 
>> Example: Convert a 2D global field of 2m air temperature from 31 December 2015 from K to degC  
<a href='http://earthserver.ecmwf.int/rasdaman/ows?service=WCS&version=2.0.1&request=ProcessCoverages&query=for c in (temp2m) return encode (c[ansi("2015-12-31T00:00")] - 273.15,"csv")' target="_blank">http://earthserver.ecmwf.int/rasdaman/ows?service=WCS&version=2.0.1&request=ProcessCoverages&query=for c in (temp2m) return encode (c[ansi("2015-12-31T00:00")] - 273.15,"csv")</a>